In [2]:
!pip install opencv-python pillow matplotlib numpy

  Using cached opencv_python-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.5 MB)


In [1]:
from __future__ import print_function, division

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2
import os

import torch
import torch.nn as nn
from torchvision import models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

ModuleNotFoundError: No module named 'matplotlib'

In [4]:
batch_size = 8

img_transform = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor(),
                                    transforms.Normalize((0.5,), (0.5,))])
train_data_dir = datasets.ImageFolder("data/train/", transform = img_transform)
train_loader = torch.utils.data.DataLoader(train_data_dir, batch_size=batch_size, shuffle=True)

# Model

In [5]:
torch.cuda.device_count()

1

In [6]:
import torchvision.models as models
import torch
import torch.nn as nn


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))

model_vgg = models.vgg16(pretrained='imagenet')

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.global_avg_pool = nn.AvgPool2d(kernel_size=7)
        self.dropout = nn.Dropout(p=0.7)
        self.fc = nn.Linear(512, 8)

    def forward(self, x):
        x = self.global_avg_pool(x)
        x = self.dropout(x)
        
        dim = x.shape[0]
        v=x.view(dim,512,-1)
        x=v.mean(2)
        x=x.view(1, dim, 512)
        x = self.fc(x)
        return x.view(-1, 8)


net = NeuralNetwork()
model = nn.Sequential(*list(model_vgg.children())[:-1])
model = nn.Sequential(model, net)
model.cuda()

Using device: cuda

NVIDIA GeForce GTX 1050


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/anis/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100.0%


Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16): M

In [7]:
input = torch.randn(1, 3, 224, 224).cuda()
output = model(input)

In [8]:
trainable_parameters = []
for name, p in model.named_parameters():
    #print(name)
    if "fc" in name:
        trainable_parameters.append(p)
optimizer = torch.optim.SGD(params=trainable_parameters, lr=0.1, momentum=1e-5)  
criterion = nn.CrossEntropyLoss()

In [9]:
total_step = len(train_loader)
loss_list = []
acc_list = []
min_loss = 9999
num_epochs = 4

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass'
        
        outputs = model(images.cuda())
       
        loss = criterion(outputs.cuda(), labels.cuda())
        loss_list.append(loss.item())
        
        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.cuda().data, 1)
        correct = (predicted.cuda() == labels.cuda()).sum().item()
        acc_list.append(correct / total)
        
        if (i % 100) == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))
    
    with open('loss.txt', 'w+') as f:
        f.write("%s\n" % loss)
    if loss < min_loss:
        min_loss = loss
        torch.save(model.state_dict(), './checkpoints/training4.pth')

Epoch [1/4], Step [1/774], Loss: 2.3335, Accuracy: 0.00%
Epoch [1/4], Step [101/774], Loss: 0.5458, Accuracy: 87.50%
Epoch [1/4], Step [201/774], Loss: 0.3680, Accuracy: 87.50%
Epoch [1/4], Step [301/774], Loss: 0.8391, Accuracy: 62.50%
Epoch [1/4], Step [401/774], Loss: 0.6640, Accuracy: 75.00%
Epoch [1/4], Step [501/774], Loss: 1.0499, Accuracy: 62.50%
Epoch [1/4], Step [601/774], Loss: 1.4799, Accuracy: 62.50%
Epoch [1/4], Step [701/774], Loss: 0.3651, Accuracy: 87.50%
Epoch [2/4], Step [1/774], Loss: 1.0342, Accuracy: 62.50%
Epoch [2/4], Step [101/774], Loss: 1.3439, Accuracy: 62.50%
Epoch [2/4], Step [201/774], Loss: 0.6661, Accuracy: 75.00%
Epoch [2/4], Step [301/774], Loss: 0.5179, Accuracy: 87.50%
Epoch [2/4], Step [401/774], Loss: 0.6059, Accuracy: 75.00%
Epoch [2/4], Step [501/774], Loss: 0.7061, Accuracy: 75.00%
Epoch [2/4], Step [601/774], Loss: 0.6539, Accuracy: 75.00%
Epoch [2/4], Step [701/774], Loss: 1.1791, Accuracy: 75.00%
Epoch [3/4], Step [1/774], Loss: 0.4952, Accu

In [15]:
from torch.autograd import Variable
import torch.nn.functional as F


model.load_state_dict(torch.load('./checkpoints/training4.pth'))       
model.eval()       
mod = nn.Sequential(*list(model_vgg.children())[:-1])
# features_blobs = []

# # def hook_feature(module, input, output):
# #     features_blobs.append(output.data.cpu().numpy())

# mod.register_forward_hook(hook_feature)

# print(features_blobs)   
params = list(net.parameters())
weight_softmax = np.squeeze(params[-2].data.cpu().numpy())

def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class -activation maps upsample to 256x256
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    
    output_cam = []
    for idx in class_idx:
#         print(weight_softmax[idx])
        cam = weight_softmax[idx].dot(feature_conv.reshape((nc, h*w)))
        
#         print(cam.shape)
#         cam = cam.view(49,7)
#         print(cam.shape)
        
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

normalize = transforms.Normalize(
  mean=[0.485, 0.456, 0.406],
  std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
  transforms.Resize((224,224)),
  transforms.ToTensor(),
  normalize
])

# list1 = [open('test1.txt','r')]
# IMG_URL = list1.readlines()
org_loc ='data/fire/'

for fname in os.listdir(org_loc):
    print(fname)
    img_pil = Image.open(org_loc + fname)
#     plt.imshow(img_pil)
    img_tensor = preprocess(img_pil)
    
    img_variable = Variable(img_tensor.unsqueeze(0))
    img_variable = img_variable.to(device)

    logit = model(img_variable)
    
    h_x = F.softmax(logit, dim=1).data.squeeze()
    
    probs, idx = h_x.sort(0, True)
    probs = probs.cpu().detach().numpy()
    idx = idx.cpu().numpy()
    
    features_blobs = mod(img_variable)
    features_blobs1 = features_blobs.cpu().detach().numpy()
    CAMs = returnCAM(features_blobs1, weight_softmax, [idx[0]])
    
    img = cv2.imread(org_loc + fname)
    
    height, width, _ = img.shape
    
    heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)
    result = heatmap * 0.3 + img * 0.5
    
    cv2.imwrite('./results/test3/' + fname, result) 

88169102_652284975535197_1304077415391166464_n.jpg
89307158_652284822201879_6446886657374289920_n.jpg
89327718_652284882201873_1111651885414088704_n.jpg
89384604_652284915535203_4330144899220373504_n.jpg
89441998_652284802201881_5384157520363257856_n.jpg
89472885_652284852201876_4462420906865590272_n.jpg
89592011_652284945535200_8137987879365246976_n.jpg
89616139_652285002201861_1795615248229597184_n.jpg
89695157_652284782201883_4420443515717156864_n.jpg
